<a href="https://colab.research.google.com/github/akbarunpam/akbarunpam/blob/main/DataMining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Download Dataset

In [ ]:
!pip install -q kaggle

1.1 Uploaded kaggle

In [ ]:
!mkdir -p ~/.kaggle
# perintah untuk membuat direktori .kaggle di direktori home (~) dari pengguna saat ini.
# Opsi -p digunakan untuk membuat direktori induk (jika belum ada) dan direktori target itu sendiri.
!cp kaggle.json ~/.kaggle/
# perintah untuk menyalin file kaggle.json ke direktori .kaggle di direktori home pengguna saat ini.
# cp adalah perintah untuk menyalin file, diikuti oleh nama file yang akan disalin (kaggle.json) dan lokasi tempat file tersebut akan disalin (~/.kaggle/).

In [ ]:
# chmod: Ini adalah perintah untuk mengubah izin file atau direktori.
# tipe permission: "r (read) 4", "w (write) 2", "x (execute) 1",
# tipe user (owner, group owner, dll)
# chmod 600: Pemilik bisa membaca dan menulis.

# ~/.kaggle/kaggle.json: Ini adalah lokasi dari file kaggle.json yang akan diubah izinnya.
!chmod 600 ~/.kaggle/kaggle.json

1.2 Processing Download

In [ ]:
# perintah untuk mengunduh dataset dengan nama diabetes-dataset yang tersedia di Kaggle dengan menggunakan Kaggle API.
# !kaggle datasets download -d akshaydattatraykhare/diabetes-dataset

# perintah untuk mengunduh dataset dengan nama body-measurements yang tersedia di Kaggle dengan menggunakan Kaggle API.
# !kaggle datasets download -d mexwell/body-measurements

!kaggle datasets download -d genshinplayer/genshin-impact-characters-stats

  0% 0.00/6.40k [00:00<?, ?B/s]
100% 6.40k/6.40k [00:00<00:00, 14.6MB/s]


In [ ]:
# import zipfile
# !mkdir /content/diabetesdataset
# with zipfile.ZipFile('diabetes-dataset.zip', 'r') as zip_ref:zip_ref.extractall('/content/')

# import zipfile
# !mkdir /content/bodydataset
# with zipfile.ZipFile('body-measurements.zip', 'r') as zip_ref:zip_ref.extractall('/content/bodydataset')

!mkdir /content/genshindataset
import zipfile
with zipfile.ZipFile('genshin-impact-characters-stats.zip', 'r') as zip_ref:zip_ref.extractall('/content/genshindataset')

# membuat direktori baru bernama Emotions, membuka file ZIP bernama emotions.zip, dan mengekstrak seluruh isi arsip ZIP ke dalam direktori yang baru dibuat tersebut.

2. Import Library

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier

In [ ]:
pwd

'/content'

3. Load & Display Dataset

In [ ]:
# membaca file CSV yang disimpan di lokasi "/content/bodydataset/bdims.csv", menyimpannya dalam sebuah DataFrame Pandas, dan kemudian menampilkan DataFrame tersebut

df = pd.read_csv("/content/genshindataset/Genshin_Impact_All_Character_Stat.csv")
# print("Semua data:\n")
# display(df)

print("1. Lima record pertama:\n")
display(df.head(2))

print("\n\n\n2. Dua record terakhir:\n")
display(df.tail(2))

print("\n\n\n3. Empat record acak:\n")
display(df.sample(2))

1. Lima record pertama:



,Character,Lv,Rarity,Element,Weapon,Main role,Ascension,Base HP,Base ATK,Base DEF
0,Amber,1,4,Pyro,Bow,Sub DPS,ATK,793,19,50
1,Amber,20,4,Pyro,Bow,Sub DPS,ATK,2038,48,129





2. Dua record terakhir:



,Character,Lv,Rarity,Element,Weapon,Main role,Ascension,Base HP,Base ATK,Base DEF
572,Kokomi,80,5,Hydro,Catalyst,Healer,Hydro DMG,12524,218,611
573,Kokomi,90,5,Hydro,Catalyst,Healer,Hydro DMG,13471,234,657





3. Empat record acak:



,Character,Lv,Rarity,Element,Weapon,Main role,Ascension,Base HP,Base ATK,Base DEF
116,Kaeya,40,4,Cryo,Sword,Support,Energy Recharge,5364,103,365
278,Xiangling,80,4,Pyro,Polearm,Sub DPS,Elemental Mastery,10122,210,623


4.1 Drp the duplicate

In [ ]:
# Check the shape before dop duplicate
df.shape

(574, 10)

In [ ]:
df.dtypes

Character    object
Lv            int64
Rarity        int64
Element      object
Weapon       object
Main role    object
Ascension    object
Base HP       int64
Base ATK      int64
Base DEF      int64
dtype: object

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 574 entries, 0 to 573
Data columns (total 10 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Character  574 non-null    object
 1   Lv         574 non-null    int64 
 2   Rarity     574 non-null    int64 
 3   Element    574 non-null    object
 4   Weapon     574 non-null    object
 5   Main role  574 non-null    object
 6   Ascension  574 non-null    object
 7   Base HP    574 non-null    int64 
 8   Base ATK   574 non-null    int64 
 9   Base DEF   574 non-null    int64 
dtypes: int64(5), object(5)
memory usage: 45.0+ KB


In [ ]:
df.describe()

,Lv,Rarity,Base HP,Base ATK,Base DEF
count,574.000000,574.000000,574.000000,574.000000,574.000000
mean,52.214286,4.512195,6860.010453,148.527875,423.303136
std,24.996567,0.500287,3325.201592,77.011853,202.560931
min,1.000000,4.000000,763.000000,8.000000,48.000000
25%,40.000000,4.000000,4405.000000,91.250000,276.000000
50%,55.000000,5.000000,6942.500000,148.000000,431.000000
75%,70.000000,5.000000,9280.000000,200.750000,575.000000
max,90.000000,5.000000,15552.000000,349.000000,922.000000


In [ ]:
df = df.drop_duplicates()